In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/MyDrive/eeg_data')
import sys
sys.path.append('/content/drive/MyDrive/eeg_data')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls ./base

Config.py		  ExractFeatures.py	       __pycache__	tensorPCA.py
config_trial_spikes.yalm  load_files_trials_spikes.py  session_data.py	utils.py
eeg.py			  MyRC_ESN.py		       subject.py


In [ ]:
!pip install mne
!pip install yasa
!pip install dotmap
!pip install nilearn
!pip install mne_connectivity
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [ ]:
# @title Script EEG Dataset Creator
import os           # Para operaciones del sistema de archivos.
import mne          # Para manejar señales EEG.
import sys          # Para interactuar con el sistema.
import argparse     # Para analizar los argumentos de la línea de comandos.
import numpy as np  # Para manipulación de matrices y operaciones numéricas.


from base.eeg import load_data, assing_channels_dt, filtering_dt
from base.eeg import all_channels, eeg_channels, eog_channels, exg_channels
from base.eeg import process_eeg_data_with_ica, process_eeg_data_without_ica
from base.eeg import assing_channels_dt, filtering_dt, extract_features_eeg_dataset


def save_data (dt_eeg, type_group, selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose = False):
    print (f'* save_data:dt_eeg :{dt_eeg}')
    if selectChannels is None: selectChannels = 'eeg'

    filename_prefix = f"dt_{type_group}_{selectChannels}_"

    if ICAflag:
        filename_prefix += "ICA_"

    if featuresFlag:
        filename_prefix += "features_"

    filename_suffix = f"{cut_low}_{decim}.npy"
    filename        = os.path.join("./dataset", filename_prefix + filename_suffix)


    # Verificar si el directorio "./data" existe, y crearlo si no
    if not os.path.exists ("dataset"):
      os.makedirs ("dataset")

    if verbose: print(f'Guardamos el fichero de datos:{filepath}')
    filepath = os.path.join (".", filename)
    np.save (filepath, dt_eeg)

def ica_extract (dataset_Younger, dataset_Older, cut_low, decim, selectChannels, channel_s, channel_z,
                       channel_d, channel_i, channel_eeg, featuresFlag, ICAflag, var_thresh, amp_thresh,
                       peak_thresh, ecg_thresh, verbose = False, trazas = True):

    selected_electrodes = {
        's': channel_s,
        'z': channel_z,
        'd': channel_d,
        'i': channel_i,
        'eeg': channel_eeg
    }.get (selectChannels, eeg_channels)

    dataset_RC_y = None
    dataset_RC_o = None

    if featuresFlag:
        if trazas: print(f' 3. Aplicamos Eliminación Artefactos y Extracción de características.' if ICAflag else f' 3. Extraccón de características sin aplicar Eliminación Artefactos')

        dataset_RC_y = extract_features_eeg_dataset (dataset_Younger, selected_electrodes, var_thresh = var_thresh,
                                                                amp_thresh = amp_thresh, peak_thresh = peak_thresh,
                                                                ecg_thresh = ecg_thresh, icaFlag = ICAflag, debug = True)
        # dataset_RC_y = np.transpose (dataset_RC_y, (0, 2, 1))
        if trazas: print(f'dataset_RC_y: {dataset_RC_y.shape}')

        dataset_RC_o = extract_features_eeg_dataset (dataset_Older, selected_electrodes, var_thresh = var_thresh,
                                                                amp_thresh = amp_thresh, peak_thresh = peak_thresh,
                                                                ecg_thresh = ecg_thresh, icaFlag = ICAflag, debug = True)
        # dataset_RC_o = np.transpose (dataset_RC_o, (0, 2, 1))
        if trazas: print(f'dataset_RC_o: {dataset_RC_o.shape}')
    else:
        print("* SIN EXTRACCION DE CARACTERISTICA")
        if ICAflag:
            print("* CON ELIMINACION ARTEFACTOS")
            if trazas: print(f' 3. Aplicamos Eliminación Artefactos')

            dataset_RC_y = process_eeg_data_with_ica (dataset_Younger, selected_electrodes, var_thresh = var_thresh,
                                                       amp_thresh = amp_thresh, ecg_thresh = ecg_thresh, peak_thresh = peak_thresh,
                                                       debug = verbose)
            if trazas: print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')

            dataset_RC_o = process_eeg_data_with_ica (dataset_Older, selected_electrodes, var_thresh = var_thresh,
                                                      amp_thresh = amp_thresh, ecg_thresh = ecg_thresh, peak_thresh = peak_thresh,
                                                      debug = verbose)
            if trazas: print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')
        else:
            print("* SIN ELIMINACION ARTEFACTOS")
            if trazas: print(f' 3. No Aplicamos Eliminación Artefactos')

            dataset_RC_y = process_eeg_data_without_ica(dataset_Younger, selected_electrodes, debug = verbose)
            if trazas: print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')

            dataset_RC_o = process_eeg_data_without_ica(dataset_Older, selected_electrodes, debug = verbose)
            if trazas: print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')

    return dataset_RC_y, dataset_RC_o

def preprocess_datasets (dataset_Younger, dataset_Older, all_channels, eeg_channels, eog_channels,
                         exg_channels, ex_type_channels, cut_low, decim, trazas = False):
    """
      Realiza el preprocesado de los conjuntos de datos EEG.

      Parámetros:
      -----------
        dataset_Younger : DataFrame
            Conjunto de datos de sujetos más jóvenes.
        dataset_Older : DataFrame
            Conjunto de datos de sujetos más mayores.
        all_channels : list
            Lista de todos los canales.
        eeg_channels : list
            Lista de canales EEG.
        eog_channels : list
            Lista de canales EOG.
        exg_channels : list
            Lista de canales EXG.
        ex_type_channels : list
            Tipos de canales EXG.
        cut_low : float
            Filtro de corte bajo para el preprocesado.
        decim : int
            Factor de decimación para el remuestreo.
        trazas : bool, opcional
            Si es True, imprime información de depuración. El valor predeterminado es False.

      Retorna:
      --------
        None
    """

    if trazas:
        print(f'* dataset_Younger: {dataset_Younger.shape}')
        print(f'* dataset_Older: {dataset_Older.shape}')

    if trazas:
        print(f' 2. Asignamos los canales y tipos de canales. tomamos los canales ex como canales: {ex_type_channels}')

    assing_channels_dt (dataset_Younger, all_channels, eeg_channels, eog_channels, exg_channels, exg_type = ex_type_channels)
    assing_channels_dt (dataset_Older, all_channels, eeg_channels, eog_channels, exg_channels, exg_type = ex_type_channels)

    print(' - Fase de preprocesado de datos')

    if trazas:
        print(f' 2. Aplicamos Filtrado: {cut_low} y remuestreo: {decim}')

    filtering_dt (dataset_Younger, cut_low = cut_low, n_decim = decim)
    filtering_dt (dataset_Older, cut_low = cut_low, n_decim = decim)

def process_datasets (dataset_Younger, dataset_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels,
                                        cut_low, decim, channel_s, channel_z, channel_d, channel_i, channel_eeg, var_thresh,
                                        amp_thresh, ecg_thresh, peak_thresh, ICAflag, featuresFlag, selectChannels,
                                        verbose, verboseGr):


  try:

    print ("Direct Younger:", dataset_Younger.iloc [0]['EEG'].get_raw ().get_data ().shape)
    print ("Direct Older:", dataset_Older.iloc [0]['EEG'].get_raw ().get_data ().shape)
    print ("Number of younger subjects:", n_y_subject)
    print ("Number of older subjects:", n_o_subject)
    print ("ex_type_channels:", ex_type_channels)
    print ("cut_low:", cut_low)
    print ("decim:", decim)
    print ("var_thresh:", var_thresh)
    print ("amp_thresh:", amp_thresh)
    print ("ecg_thresh:", ecg_thresh)
    print ("peak_thresh:", peak_thresh)
    print ("selectChannel:", selectChannels)
    print ("ICA flag:", ICAflag)
    print ("features flag:", featuresFlag)
    print ("rand_load:", rand_load)
    print ("verbose:", verbose)
    print ("verbose:", verboseGr)

    print (' - Iniciamos la Carga de Datos y el Script')



    dataset_RC_y, dataset_RC_o = ica_extract (dataset_Younger, dataset_Older, cut_low, decim, selectChannels, channel_s, channel_z,
                                                      channel_d, channel_i, channel_eeg, featuresFlag, ICAflag, var_thresh, amp_thresh,
                                                      peak_thresh, ecg_thresh, verbose = False)

    print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')
    print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')

    print ('- Guardamos los datos en el directorio ./dataset')

    if verbose: print (f' 4.1. Guardamos los datos EEG preprocesados de Younger')
    save_data(dataset_RC_y, "Younger", selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose)

    if verbose: print (f' 4.1. Guardamos los datos EEG preprocesados de Older')
    save_data (dataset_RC_o, "Older", selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose)

  except ValueError as e:
    print(f"Error para el Scritp: {e}")

def generate_datasets (direct_Younger, direct_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels, selectChannels,
                      cut_low, decim, var_thresh, amp_thresh, ecg_thresh, peak_thresh, ICAflag, featuresFlag, verbose, verboseGr):

    channel_s = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'Fz', 'AFz',
                       'C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'O1', 'O2', 'Oz']

    channel_z = [ 'Oz', 'POz','Pz', 'CPz', 'Fpz','AFz', 'Fz', 'FCz', 'Cz'] # 'Iz'

    channel_d = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1',  'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz']  # 'Iz'
    channel_i = [ 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']  # 'Iz'
    channel_eeg = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']
    eeg_channels = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

    selectChannels_options = ['s','z', 'd', 'i','eeg']
    ICAflag_options        = [True, False]
    feature_options        = [True, False]

    if verbose: print (f' 1. Cargamos los datos del numero de sjetos jovenes:{n_y_subject} y Mayore:{n_o_subject}')
    # Directorios de jóvenes y mayores
    direct_Younger = './Younger'  # Reemplazar 'Younger' con la ruta al directorio de los jóvenes
    direct_Older   = './Older'    # Reemplazar 'Older' con la ruta al directorio de los mayores

    dataset_Younger, dataset_Older = load_data (direct_Younger = direct_Younger,direct_Older = direct_Older,
                                                 n_y_subject = n_y_subject, n_o_subject = n_o_subject, rand_load = rand_load,
                                                 verbose = verbose, verbose_gr = verboseGr)

    print (f'dataset_Younger.shape: {dataset_Younger.shape}')
    print (f'dataset_Older.shape: {dataset_Older.shape}')

    # Adignamos tipos de canales y filtramos las señales
    preprocess_datasets (dataset_Younger, dataset_Older, all_channels, eeg_channels, eog_channels,
                         exg_channels, ex_type_channels, cut_low, decim, trazas = True)


    print (f"**** Processing with selectChannels = {selectChannels} and ICAflag = {ICAflag} and featureFlag:{featuresFlag}")
    process_datasets (dataset_Younger, dataset_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels,
                      cut_low, decim, channel_s, channel_z, channel_d, channel_i, channel_eeg, var_thresh, amp_thresh,
                      ecg_thresh, peak_thresh, ICAflag, featuresFlag, selectChannels, verbose, verboseGr)
def main (args):
  generate_datasets (args.direct_Younger, args.direct_Older, args.n_y_subject, args.n_o_subject, args.rand_load,
                       args.ex_type_channels, args.selectChannels, args.cut_low, args.decim, args.var_thresh, args.amp_thresh,
                       args.ecg_thresh, args.peak_thresh, args.icaFlag, args.featuresFlag, args.verbose, args.verboseGr)

if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
          args = argparse.Namespace (
                      direct_Younger   = './Younger',
                      direct_Older     = './Older',
                      n_y_subject      = 23,
                      n_o_subject      = 24,
                      ex_type_channels = 'ecg',
                      cut_low          = 30,
                      decim            = 4,
                      var_thresh       = 1.5,
                      amp_thresh       = 1.5,
                      ecg_thresh       = 0.25,
                      peak_thresh      = 0.85,
                      selectChannels   = 's', # s, z, d, i, eeg
                      rand_load        = True,
                      icaFlag          = True,
                      featuresFlag     = True,
                      verbose          = False,
                      verboseGr        = False
          )
    else:
        parser = argparse.ArgumentParser (description = 'Generar conjuntos de datos para todas las combinaciones de selectChannels e ICAflag.')

        parser.add_argument ('--direct_Younger',   type = str,   default = './Younger', help = 'Directorio para sujetos jóvenes')
        parser.add_argument ('--direct_Older',     type = str,   default = './Older',   help = 'Directorio para sujetos mayores')
        parser.add_argument ('--n_y_subject',      type = int,   default = 23,          help = 'Número de sujetos jóvenes')
        parser.add_argument ('--n_o_subject',      type = int,   default = 24,          help = 'Número de sujetos mayores')
        parser.add_argument ('--ex_type_channels', type = str,   default = 'ecg',       help = 'Tipo de canal para los canales externos (EX_)')
        parser.add_argument ('--cut_low',          type = int,   default = 30,          help = 'Frecuencia corte Paso Bajo')
        parser.add_argument ('--decim',            type = int,   default = 4,           help = 'Remuestreo de las señales EEG')
        parser.add_argument ('--var_thresh',       type = float, default = 1.5,         help = 'Eliminación artefactos por z-score')
        parser.add_argument ('--amp_thresh',       type = float, default = 1.5,         help = 'Eliminación artefactos por amplitud')
        parser.add_argument ('--ecg_thresh',       type = float, default = 0.25,        help = 'Eliminación artefactos por canales')
        parser.add_argument ('--peak_thresh',      type = float, default = 0.85,        help = 'Eliminación de artefactos por picos')
        parser.add_argument ('--selectChannels',   type = str,   default = 's',         help = 'Si queremos realizar el estudio solo con los canales eeg seleccionados (s, z, i, d)')
        parser.add_argument ('--rand_load',        action = 'store_true', default = True,  help = 'Carga aleatoria de los ficheros de datos')
        parser.add_argument ('--icaFlag',          action = 'store_true', default = False, help = 'Realizamos eliminación de artefactos')
        parser.add_argument ('--featuresFlag',     action = 'store_true', default = False, help = 'Realizamos extracción características de las señales')
        parser.add_argument ('--verbose',          action = 'store_true', default = False, help = 'Activar trazas')
        parser.add_argument ('--verboseGr',        action = 'store_true', default = False, help = 'Activar trazas con Gráficos')


        args = parser.parse_args ()

    main (args)

None
Creating RawArray with float64 data, n_channels=73, n_times=166400
    Range : 0 ... 166399 =      0.000 ...   324.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=165376
    Range : 0 ... 165375 =      0.000 ...   322.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=160768
    Range : 0 ... 160767 =      0.000 ...   313.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=164352
    Range : 0 ... 164351 =      0.000 ...   320.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=164352
    Range : 0 ... 164351 =      0.000 ...   320.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=166912
    Range : 0 ... 166911 =      0.000 ...   325.998 secs
Ready.
None
Creating RawArray with float64 data, n_channels=73, n_times=194560
    Range : 0 ... 194559 =      0.000 ...   379.998 secs
Ready.
None
Creating RawArray with float64 data, n_chan

In [ ]:
 # @title Script EEG Dataset Creator

import os           # Para operaciones del sistema de archivos.
import mne          # Para manejar señales EEG.
import sys          # Para interactuar con el sistema.
import argparse     # Para analizar los argumentos de la línea de comandos.
import numpy as np  # Para manipulación de matrices y operaciones numéricas.


from base.eeg import load_data, assing_channels_dt, filtering_dt
from base.eeg import all_channels, eeg_channels, eog_channels, exg_channels
from base.eeg import process_eeg_data_with_ica, process_eeg_data_without_ica
from base.eeg import assing_channels_dt, filtering_dt, extract_features_eeg_dataset


def save_data (dt_eeg, type_group, selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose = False):
    print (f'* save_data:dt_eeg :{dt_eeg}')
    if selectChannels is None: selectChannels = 'eeg'

    filename_prefix = f"dt_{type_group}_{selectChannels}_"

    if ICAflag:
        filename_prefix += "ICA_"

    if featuresFlag:
        filename_prefix += "features_"

    filename_suffix = f"{cut_low}_{decim}.npy"
    filename        = os.path.join("./dataset", filename_prefix + filename_suffix)


    # Verificar si el directorio "./data" existe, y crearlo si no
    if not os.path.exists ("dataset"):
      os.makedirs ("dataset")

    if verbose: print(f'Guardamos el fichero de datos:{filepath}')
    filepath = os.path.join (".", filename)
    np.save (filepath, dt_eeg)

def ica_extract (dataset_Younger, dataset_Older, cut_low, decim, selectChannels, channel_s, channel_z,
                       channel_d, channel_i, channel_eeg, featuresFlag, ICAflag, var_thresh, amp_thresh,
                       peak_thresh, ecg_thresh, verbose = False, trazas = True):

    selected_electrodes = {
        's': channel_s,
        'z': channel_z,
        'd': channel_d,
        'i': channel_i,
        'eeg': channel_eeg
    }.get (selectChannels, eeg_channels)

    dataset_RC_y = None
    dataset_RC_o = None

    if featuresFlag:
        if trazas: print(f' 3. Aplicamos Eliminación Artefactos y Extracción de características.' if ICAflag else f' 3. Extraccón de características sin aplicar Eliminación Artefactos')

        dataset_RC_y = extract_features_eeg_dataset (dataset_Younger, selected_electrodes, var_thresh = var_thresh,
                                                                amp_thresh = amp_thresh, peak_thresh = peak_thresh,
                                                                ecg_thresh = ecg_thresh, icaFlag = ICAflag, debug = True)
        dataset_RC_y = np.transpose (dataset_RC_y, (0, 2, 1))
        if trazas: print(f'dataset_RC_y: {dataset_RC_y.shape}')

        dataset_RC_o = extract_features_eeg_dataset (dataset_Older, selected_electrodes, var_thresh = var_thresh,
                                                                amp_thresh = amp_thresh, peak_thresh = peak_thresh,
                                                                ecg_thresh = ecg_thresh, icaFlag = ICAflag, debug = True)
        dataset_RC_o = np.transpose (dataset_RC_o, (0, 2, 1))
        if trazas: print(f'dataset_RC_o: {dataset_RC_o.shape}')
    else:
        print("* SIN EXTRACCION DE CARACTERISTICA")
        if ICAflag:
            print("* CON ELIMINACION ARTEFACTOS")
            if trazas: print(f' 3. Aplicamos Eliminación Artefactos')

            dataset_RC_y = process_eeg_data_with_ica (dataset_Younger, selected_electrodes, var_thresh = var_thresh,
                                                       amp_thresh = amp_thresh, ecg_thresh = ecg_thresh, peak_thresh = peak_thresh,
                                                       debug = verbose)
            if trazas: print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')

            dataset_RC_o = process_eeg_data_with_ica (dataset_Older, selected_electrodes, var_thresh = var_thresh,
                                                      amp_thresh = amp_thresh, ecg_thresh = ecg_thresh, peak_thresh = peak_thresh,
                                                      debug = verbose)
            if trazas: print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')
        else:
            print("* SIN ELIMINACION ARTEFACTOS")
            if trazas: print(f' 3. No Aplicamos Eliminación Artefactos')

            dataset_RC_y = process_eeg_data_without_ica(dataset_Younger, selected_electrodes, debug = verbose)
            if trazas: print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')

            dataset_RC_o = process_eeg_data_without_ica(dataset_Older, selected_electrodes, debug = verbose)
            if trazas: print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')

    return dataset_RC_y, dataset_RC_o

def preprocess_datasets (dataset_Younger, dataset_Older, all_channels, eeg_channels, eog_channels,
                         exg_channels, ex_type_channels, cut_low, decim, trazas = False):
    """
      Realiza el preprocesado de los conjuntos de datos EEG.

      Parámetros:
      -----------
        dataset_Younger : DataFrame
            Conjunto de datos de sujetos más jóvenes.
        dataset_Older : DataFrame
            Conjunto de datos de sujetos más mayores.
        all_channels : list
            Lista de todos los canales.
        eeg_channels : list
            Lista de canales EEG.
        eog_channels : list
            Lista de canales EOG.
        exg_channels : list
            Lista de canales EXG.
        ex_type_channels : list
            Tipos de canales EXG.
        cut_low : float
            Filtro de corte bajo para el preprocesado.
        decim : int
            Factor de decimación para el remuestreo.
        trazas : bool, opcional
            Si es True, imprime información de depuración. El valor predeterminado es False.

      Retorna:
      --------
        None
    """

    if trazas:
        print(f'* dataset_Younger: {dataset_Younger.shape}')
        print(f'* dataset_Older: {dataset_Older.shape}')

    if trazas:
        print(f' 2. Asignamos los canales y tipos de canales. tomamos los canales ex como canales: {ex_type_channels}')

    assing_channels_dt (dataset_Younger, all_channels, eeg_channels, eog_channels, exg_channels, exg_type = ex_type_channels)
    assing_channels_dt (dataset_Older, all_channels, eeg_channels, eog_channels, exg_channels, exg_type = ex_type_channels)

    print(' - Fase de preprocesado de datos')

    if trazas:
        print(f' 2. Aplicamos Filtrado: {cut_low} y remuestreo: {decim}')

    filtering_dt (dataset_Younger, cut_low = cut_low, n_decim = decim)
    filtering_dt (dataset_Older, cut_low = cut_low, n_decim = decim)

def process_datasets (dataset_Younger, dataset_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels,
                                        cut_low, decim, channel_s, channel_z, channel_d, channel_i, channel_eeg, var_thresh,
                                        amp_thresh, ecg_thresh, peak_thresh, ICAflag, featuresFlag, selectChannels,
                                        verbose, verboseGr):


  try:

    print ("Direct Younger:", dataset_Younger.iloc [0]['EEG'].get_raw ().get_data ().shape)
    print ("Direct Older:", dataset_Older.iloc [0]['EEG'].get_raw ().get_data ().shape)
    print ("Number of younger subjects:", n_y_subject)
    print ("Number of older subjects:", n_o_subject)
    print ("ex_type_channels:", ex_type_channels)
    print ("cut_low:", cut_low)
    print ("decim:", decim)
    print ("var_thresh:", var_thresh)
    print ("amp_thresh:", amp_thresh)
    print ("ecg_thresh:", ecg_thresh)
    print ("peak_thresh:", peak_thresh)
    print ("selectChannel:", selectChannels)
    print ("ICA flag:", ICAflag)
    print ("features flag:", featuresFlag)
    print ("rand_load:", rand_load)
    print ("verbose:", verbose)
    print ("verbose:", verboseGr)

    print (' - Iniciamos la Carga de Datos y el Script')



    dataset_RC_y, dataset_RC_o = ica_extract (dataset_Younger, dataset_Older, cut_low, decim, selectChannels, channel_s, channel_z,
                                                      channel_d, channel_i, channel_eeg, featuresFlag, ICAflag, var_thresh, amp_thresh,
                                                      peak_thresh, ecg_thresh, verbose = False)

    print(f'dataset_RC_y.shape: {dataset_RC_y.shape}')
    print(f'dataset_RC_o.shape: {dataset_RC_o.shape}')

    print ('- Guardamos los datos en el directorio ./dataset')

    if verbose: print (f' 4.1. Guardamos los datos EEG preprocesados de Younger')
    save_data(dataset_RC_y, "Younger", selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose)

    if verbose: print (f' 4.1. Guardamos los datos EEG preprocesados de Older')
    save_data (dataset_RC_o, "Older", selectChannels, cut_low, decim, ICAflag, featuresFlag, verbose)

  except ValueError as e:
    print(f"Error para el Scritp: {e}")

def generate_datasets (direct_Younger, direct_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels, selectChannels,
                      cut_low, decim, var_thresh, amp_thresh, ecg_thresh, peak_thresh, ICAflag, featuresFlag, verbose, verboseGr):

    channel_s = ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'Fz', 'AFz',
                       'C3', 'C4', 'Cz', 'P3', 'P4', 'Pz', 'O1', 'O2', 'Oz']

    channel_z = [ 'Oz', 'POz','Pz', 'CPz', 'Fpz','AFz', 'Fz', 'FCz', 'Cz'] # 'Iz'

    channel_d = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1',  'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz']  # 'Iz'
    channel_i = [ 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']  # 'Iz'
    channel_eeg = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']
    eeg_channels = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7',
                  'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7',
                  'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9',
                  'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz',
                  'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4',
                  'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz',
                  'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2',
                  'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

    selectChannels_options = ['s','z', 'd', 'i','eeg']
    ICAflag_options        = [True, False]
    feature_options        = [True, False]

    if verbose: print (f' 1. Cargamos los datos del numero de sjetos jovenes:{n_y_subject} y Mayore:{n_o_subject}')
    # Directorios de jóvenes y mayores
    direct_Younger = './Younger'  # Reemplazar 'Younger' con la ruta al directorio de los jóvenes
    direct_Older   = './Older'    # Reemplazar 'Older' con la ruta al directorio de los mayores

    dataset_Younger, dataset_Older = load_data (direct_Younger = direct_Younger,direct_Older = direct_Older,
                                                 n_y_subject = n_y_subject, n_o_subject = n_o_subject, rand_load = rand_load,
                                                 verbose = verbose, verbose_gr = verboseGr)

    print (f'dataset_Younger.shape: {dataset_Younger.shape}')
    print (f'dataset_Older.shape: {dataset_Older.shape}')

    # Adignamos tipos de canales y filtramos las señales
    preprocess_datasets (dataset_Younger, dataset_Older, all_channels, eeg_channels, eog_channels,
                         exg_channels, ex_type_channels, cut_low, decim, trazas = True)

    for selectChannels in selectChannels_options:
        for ICAflag in ICAflag_options:
          for featuresFlag in feature_options:
            print (f"**** Processing with selectChannels = {selectChannels} and ICAflag = {ICAflag} and featureFlag:{featuresFlag}")
            process_datasets (dataset_Younger, dataset_Older, n_y_subject, n_o_subject, rand_load, ex_type_channels,
                             cut_low, decim, channel_s, channel_z, channel_d, channel_i, channel_eeg, var_thresh, amp_thresh,
                              ecg_thresh, peak_thresh, ICAflag, featuresFlag, selectChannels, verbose, verboseGr)
def main (args):
  generate_datasets (args.direct_Younger, args.direct_Older, args.n_y_subject, args.n_o_subject, args.rand_load,
                       args.ex_type_channels, args.selectChannels, args.cut_low, args.decim, args.var_thresh, args.amp_thresh,
                       args.ecg_thresh, args.peak_thresh, args.icaFlag, args.featuresFlag, args.verbose, args.verboseGr)

if __name__ == "__main__":
    if 'ipykernel' in sys.modules:
          args = argparse.Namespace (
                      direct_Younger   = './Younger',
                      direct_Older     = './Older',
                      n_y_subject      = 23,
                      n_o_subject      = 24,
                      ex_type_channels = 'ecg',
                      cut_low          = 30,
                      decim            = 4,
                      var_thresh       = 1.5,
                      amp_thresh       = 1.5,
                      ecg_thresh       = 0.25,
                      peak_thresh      = 0.85,
                      selectChannels   = 's', # s, z, d, i, eeg
                      rand_load        = True,
                      icaFlag          = True,
                      featuresFlag     = True,
                      verbose          = False,
                      verboseGr        = False
          )
    else:
        parser = argparse.ArgumentParser (description = 'Generar conjuntos de datos para todas las combinaciones de selectChannels e ICAflag.')

        parser.add_argument ('--direct_Younger',   type = str,   default = './Younger', help = 'Directorio para sujetos jóvenes')
        parser.add_argument ('--direct_Older',     type = str,   default = './Older',   help = 'Directorio para sujetos mayores')
        parser.add_argument ('--n_y_subject',      type = int,   default = 23,          help = 'Número de sujetos jóvenes')
        parser.add_argument ('--n_o_subject',      type = int,   default = 24,          help = 'Número de sujetos mayores')
        parser.add_argument ('--ex_type_channels', type = str,   default = 'ecg',       help = 'Tipo de canal para los canales externos (EX_)')
        parser.add_argument ('--cut_low',          type = int,   default = 30,          help = 'Frecuencia corte Paso Bajo')
        parser.add_argument ('--decim',            type = int,   default = 4,           help = 'Remuestreo de las señales EEG')
        parser.add_argument ('--var_thresh',       type = float, default = 1.5,         help = 'Eliminación artefactos por z-score')
        parser.add_argument ('--amp_thresh',       type = float, default = 1.5,         help = 'Eliminación artefactos por amplitud')
        parser.add_argument ('--ecg_thresh',       type = float, default = 0.25,        help = 'Eliminación artefactos por canales')
        parser.add_argument ('--peak_thresh',      type = float, default = 0.85,        help = 'Eliminación de artefactos por picos')
        parser.add_argument ('--selectChannels',   type = str,   default = 's',         help = 'Si queremos realizar el estudio solo con los canales eeg seleccionados (s, z, i, d)')
        parser.add_argument ('--rand_load',        action = 'store_true', default = True,  help = 'Carga aleatoria de los ficheros de datos')
        parser.add_argument ('--icaFlag',          action = 'store_true', default = False, help = 'Realizamos eliminación de artefactos')
        parser.add_argument ('--featuresFlag',     action = 'store_true', default = False, help = 'Realizamos extracción características de las señales')
        parser.add_argument ('--verbose',          action = 'store_true', default = False, help = 'Activar trazas')
        parser.add_argument ('--verboseGr',        action = 'store_true', default = False, help = 'Activar trazas con Gráficos')


        args = parser.parse_args ()

    main (args)

Se han truncado las últimas 5000 líneas del flujo de salida.
  [-3.10971985e-07 -8.99776284e-07  6.64943461e-07 ... -6.60754319e-07
    2.84364792e-07 -4.98249251e-07]
  [ 2.62358131e-07 -7.60848858e-07  1.33041455e-06 ...  2.54248888e-08
    6.30289589e-07  3.04252064e-08]
  ...
  [-3.01815218e-06 -5.08449732e-06 -3.24104033e-06 ... -9.34339366e-06
   -3.12003581e-06 -5.82987265e-06]
  [-2.18630023e-06 -4.03073382e-06 -2.95243194e-06 ... -1.10658024e-05
   -2.68691267e-06 -6.38301136e-06]
  [-2.79816320e-08 -2.15369624e-06 -1.50606723e-06 ... -1.04166704e-05
   -1.57280038e-06 -5.31881470e-06]]

 [[-3.86304480e-07 -3.34171400e-07 -2.38827870e-07 ... -2.59741848e-07
   -2.57074842e-07 -1.43773690e-07]
  [-1.47915846e-06 -4.45176578e-07 -7.54019739e-07 ... -1.21653164e-06
   -1.01577993e-06  7.76869392e-07]
  [-8.82082479e-07  1.02986548e-07 -2.36218625e-07 ... -5.66746732e-07
   -5.26253299e-07  6.29924046e-07]
  ...
  [-1.23449888e-06 -1.12679068e-07 -7.14876554e-07 ... -1.16709797e-0

In [ ]:
# @title Listado ficheros datasets creados de los datos raw
import os
import numpy as np

def leer_ficheros_y_mostrar_dimensiones(directorio='dataset'):
    # Verifica si el directorio existe
    if not os.path.exists(directorio):
        print(f"El directorio {directorio} no existe.")
        return

    # Obtiene la lista de archivos en el subdirectorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.npy')]

    # Verifica si hay archivos en el directorio
    if not archivos:
        print(f"No se encontraron archivos .npy en el directorio {directorio}.")
        return

    # Lee cada archivo y muestra sus dimensiones
    for archivo in archivos:
        ruta_archivo = os.path.join(directorio, archivo)
        try:
            datos = np.load(ruta_archivo)
            print(f"El archivo {archivo} tiene dimensiones: {datos.shape}")
        except Exception as e:
            print(f"No se pudo leer el archivo {archivo}: {e}")

# Llamada a la función
leer_ficheros_y_mostrar_dimensiones()

El archivo dt_Younger_s_ICA_features_30_4.npy tiene dimensiones: (23, 360, 31)
El archivo dt_Older_s_ICA_features_30_4.npy tiene dimensiones: (24, 360, 31)
El archivo dt_Younger_s_ICA_30_4.npy tiene dimensiones: (23, 39680, 17)
El archivo dt_Older_s_ICA_30_4.npy tiene dimensiones: (24, 39680, 17)
El archivo dt_Younger_s_features_30_4.npy tiene dimensiones: (23, 360, 31)
El archivo dt_Older_s_features_30_4.npy tiene dimensiones: (24, 360, 31)
El archivo dt_Younger_s_30_4.npy tiene dimensiones: (23, 39680, 17)
El archivo dt_Older_s_30_4.npy tiene dimensiones: (24, 39680, 17)
El archivo dt_Younger_z_ICA_features_30_4.npy tiene dimensiones: (23, 296, 31)
El archivo dt_Older_z_ICA_features_30_4.npy tiene dimensiones: (24, 296, 31)
El archivo dt_Younger_z_ICA_30_4.npy tiene dimensiones: (23, 39680, 9)
El archivo dt_Older_z_ICA_30_4.npy tiene dimensiones: (24, 39680, 9)
El archivo dt_Younger_z_features_30_4.npy tiene dimensiones: (23, 296, 31)
El archivo dt_Older_z_features_30_4.npy tiene dim